# KNN

Este código implementa uma versão simples do algoritmo de classificação KNN. Por "simples" quero dizer que há implementações mais eficientes!

In [2]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split

In [5]:
#vote_histogram: esta função faz a contagem das classes dos k vizinhos mais próximos.
#Já recebe o vetor com os k mais próximos.
def class_histogram(kn_neighbors, n_classes):
    #Cria um vetor c zerado com n_classes posições
    c = [0] * n_classes
    #Para cada vizinho entre os k mais próximos
    for i in kn_neighbors:
        #incrementar contagem
        c[i]+=1
    #retornar contagem das classes dos k vizinhos mais próximos
    return c

#Faz a classificação do exemplo x baseado nos k mais próximos em X_train.
def knn(X_train, Y_train, x, k):
    #Calcula o vetor de distâncias entre x e todos os pontos em X_train
    d = euclidean_distances(x.reshape(1,-1), X_train).reshape(-1)
    #Ordena o vetor d e retorna os índices da ordenação em relação ao vetor d original. Não mexe no vetor d.
    #Isto ajuda pq é necessário indexar Y_train das posições correspondentes depois da ordenação!
    idx = np.argsort(d)
    #Calcula a contagem das classes dos k vizinhos mais próximos:
    #    Y_train[idx][:k] <-- pega os rótulos dos k vizinhos mais próximos!
    hist = class_histogram(Y_train[idx][:k], len(set(Y_train)))
    #hist apenas é um vetor [c0, c1, c2, ..., c_nclasses] de forma que c0 é a quantidade de vizinhos mais próximos
    #que são da classe 0, c1 é a quantidade de vizinhos mais proximos que são da classe 1, e assim por diante.
    
    #Conforme vimos, o knn classifica x como sendo da classe com a maior quantidade de vizinhos mais próximos.
    #assim, basta retornar a posição da classe que tem a maior quantidade de vizinhos mais próximos!
    #    ex: se np.argmax(hist) == 0, a classe 0 tem a maioria dos vizinhos mais próximos de x.
    return np.argmax(hist)

def knn_2():
    #Uma das formas de acelerar é passar o X_test todo. Acredito que a implementação do euclidean_distances
    #é eficiente para computar tudo de uma vez (com implementação vetorial)
    raise NotImplementedError


In [27]:
#Carrega o toy dataset ``digits'' do sklearn. Esse dataset é composto de 1797 imagens de dígitos manuscritos 8x8 em 
#16 tons de cinza. Há 64 características por imagem, uma para o valor de cada pixel.
X, Y = load_digits(return_X_y=True)

#Vamos dividir o dataset com 80% no treino e 20% no teste.
#X_train é o conjunto de treino com os exemplos e Y_train são os gabaritos de cada exemplo de X_train.
#X_test é o conjunto de treino com os exemplos e Y_test são os gabaritos de cada exemplo de X_test
#se shuffle=True, "embaralha" o dataset antes de fazer a separação.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

misses = 0
hits = 0
k = 3

#Para cada exemplo no conjunto de teste
for i, x in enumerate(X_test):
    #Classificar o exemplo. Se o exemplo estiver correto (for igual do gabarito)
    if knn(X_train, Y_train, x, k) == Y_test[i]:
        #incrementar os acertos
        hits+=1
    else:
        #Senão, incrementar os erros
        misses+=1

#A acurácia é dada por acertos / (acertos + erros).
print ("acurácia: %.02f" % (hits / float(hits + misses)))

acurácia: 0.99


# Validação cruzada para otimização do hiperparâmetro $k$

Note que $k$ é um parâmetro do classificador KNN, ou seja, $k$ é um valor que deve ser escolhido para que o KNN funcione. Entretanto, este parâmetro tem impacto direto no resultado da classificação.

Como escolher $k$ para que o sistema obtenha o melhor resultado possível?

Não existe nenhum mecanismo que possa garantir isso, mas um mecanismo amplamente utilizado para este fim é o uso de validação cruzada.

A idéia é simples: devemos testar o classificador variando o valor de $k$, e depois devemos escolher o $k$ que obteve o melhor resultado para usar no mundo real. No entanto, não é aconselhado testar os diferentes valores de $k$ com o conjunto de teste, uma vez que estaria "tunando" o modelo para o conjunto de teste! Desta forma, o resultado não seria confiável para extrapolar para avaliar o comportamento do sistema no mundo real. Assim, uma das idéias é dividir o conjunto de treino em dois conjuntos: treino e validação. Desta forma, o conjunto de treino é sempre usado para treinar o modelo, mas a avaliação dos hiperparâmetros ($k$, no caso do KNN) é feita no conjunto de validação! O desempenho do sistema é avaliado no conjunto de teste, usando os melhores hiperparametros encontrados no conjunto de validação.

Veja o código-exemplo abaixo, que implementa a validação cruzada para otimização do hiperparametro $k$ do KNN. Os comentários apresentados no exemplo anterior foram removidos, e somente comentários relacionados ao novo procedimento são mostrados.

In [37]:
X, Y = load_digits(return_X_y=True)

#Vamos dividir o dataset com 80% no treino e 20% no teste.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
#E agora vamos dividir o conjunto de treino em 80% para treino, e 20% para validação.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=False)

#A lista de ks a serem testados
ks = [1, 3, 5, 7, 9]
accs = []

#Testar o conjunto de validação para todos os ks candidatos!
for k in ks:
    misses = 0
    hits = 0    

    #Para cada exemplo no conjunto de validação
    for i, x in enumerate(X_val):
        #Classificar o exemplo. Se o exemplo estiver correto (for igual do gabarito)
        if knn(X_train, Y_train, x, k) == Y_val[i]:
            #incrementar os acertos
            hits+=1
        else:
            #Senão, incrementar os erros
            misses+=1
    #salvar a acurácia do modelo com o valor atual de k
    accs.append((hits / float(hits + misses)))
    print ("acurácia p/ k = %d: %.02f" % (k, accs[-1]))

#Escolher o k cuja acurácia foi máxima no conjunto de validação!
k = ks[np.argmax(accs)]

print ("melhor k: %d" % k)

misses = 0
hits = 0    

#Para cada exemplo no conjunto de teste.
for i, x in enumerate(X_test):
    #Classificar o exemplo. Se o exemplo estiver correto (for igual do gabarito)
    #Note que o k escolhido é aquele cuja acurácia foi máxima no conjunto de validação!
    #Não usamos o conjunto de teste até agora!
    if knn(X_train, Y_train, x, k) == Y_test[i]:
        #incrementar os acertos
        hits+=1
    else:
        #Senão, incrementar os erros
        misses+=1

#A acurácia é dada por acertos / (acertos + erros).
print ("acurácia no teste p/ k = %d: %.02f" % (k, hits / float(hits + misses)))

acurácia p/ k = 1: 0.99
acurácia p/ k = 3: 0.99
acurácia p/ k = 5: 0.98
acurácia p/ k = 7: 0.98
acurácia p/ k = 9: 0.98
melhor k: 3
acurácia no teste p/ k = 3: 0.96
